In [1]:
import re
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score,make_scorer
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.model_selection import ShuffleSplit,cross_val_score

In [2]:
#Read Train data
with open("/content/drive/MyDrive/Train.txt", "r") as tr:
    Train = tr.readlines()
#read Test data
with open("/content/drive/MyDrive/Test.txt", "r") as te:
    Test = te.readlines()
print (len(Train))


800


In [3]:
drug_active = []
train_features = []
test_features = []


for i in range(len(Train)):
    lines = Train[i]
    for l in range(len(lines)):
        if l == 1:
            drug_active.append(lines[0:1])

#Split the train dataset features into list
for features in Train:       
    features = features.replace("0\t", "")
    features = features.replace("1\t", "")
    features = features.replace("\n", "")
    features = features.split()       
    train_features.append(features)

#Split the test dataset features into list
for features in Test:       
    features = features.replace("0\t", "")
    features = features.replace("1\t", "")
    features = features.replace("\n", "")
    features = features.split()       
    test_features.append(features)

print(len(train_features))
print(len(test_features))
print(len(drug_active))

800
350
800


In [4]:
train_sparse = []
total_features_count = 100001

for features in train_features:
  train_sparse_set = [0]*total_features_count
  for feature in features:
    train_sparse_set[int(feature)] = 1
  train_sparse.append(train_sparse_set)

test_sparse = []

for features in test_features:
  test_sparse_set = [0]*total_features_count
  for feature in features:
    test_sparse_set[int(feature)] = 1
  test_sparse.append(test_sparse_set)

print(len(train_sparse[0]))
x_train, x_test, y_train, y_test = train_test_split(train_sparse,drug_active,test_size=0.20,random_state=42)

100001


In [5]:
chi2_selection = SelectKBest(chi2, k=300)
train_sparse_1=chi2_selection.fit_transform(x_train,y_train)
test_sparse_1 = chi2_selection.transform(x_test)
train_sparse_final=chi2_selection.fit_transform(train_sparse,drug_active)
test_sparse_final = chi2_selection.transform(test_sparse)
print(train_sparse_1.shape)
print(test_sparse_1.shape)

(640, 300)
(160, 300)


In [6]:
smote_sm = SMOTE(random_state = 42)
train_sparse_sm_1,drug_active_sm_1 = smote_sm.fit_resample(train_sparse_final,drug_active)
train_sparse_sm,drug_active_sm = smote_sm.fit_resample(x_train,y_train)
print(len(train_sparse_sm))
print(len(train_sparse_sm[0]))


1152
100001


In [7]:
svd = TruncatedSVD(n_components=350)
svd1 = TruncatedSVD(n_components=350)

pca1 = PCA(n_components = 0.95)
sparse_train_svd = svd1.fit_transform(train_sparse,drug_active)
train_svd1 =  svd1.transform(x_train)
test_svd1 = svd1.transform(x_test)
sparse_test_svd = svd1.transform(test_sparse)

sparse_train_pca = pca1.fit_transform(train_sparse,drug_active)
train_pca1 =  pca1.transform(x_train)
test_pca1 = pca1.transform(x_test)
sparse_test_pca = pca1.transform(test_sparse)


sparse_svd = svd.fit_transform(train_sparse_sm,drug_active_sm)
pca = PCA(n_components = 0.95)
sparse_pca = pca.fit_transform(train_sparse_sm)


train_svd = svd.transform(x_train)
#print(train_svd)
train_pca = pca.transform(x_train)
print(train_pca)


test_svd = svd.transform(x_test)
#print(test_svd)
test_pca = pca.transform(x_test)
#print(test_pca)

test_sparse_svd = svd.transform(test_sparse)
test_sparse_pca = pca.transform(test_sparse)

#print(sparse_svd.shape)
print(train_svd.shape)
print(test_svd.shape)
print(test_sparse_svd.shape)
#print(test_sparse_svd.shape)
#print(sparse_pca.shape)
#print(train_pca.shape)
#print(test_pca.shape)
#print(test_sparse_pca.shape)
print(svd.explained_variance_ratio_.sum())
print(svd1.explained_variance_ratio_.sum())

[[ 0.86343415 -0.9389797  -1.8621428  ... -0.07786184 -1.40887876
   0.27059813]
 [ 0.23559506 -0.46630539 -1.99946801 ...  1.12678214  0.73971695
  -0.70937452]
 [ 0.22928774 -0.56249729 -1.77969048 ...  0.07219735 -0.47950398
  -0.16373381]
 ...
 [ 0.49479459 -0.65691126 -2.31260623 ...  0.50008196  0.43206119
   1.51974749]
 [ 1.56153352 -1.6991631  -1.38846366 ...  0.72498023  0.69248242
  -0.28803238]
 [ 0.25934995 -0.33730987 -2.44705785 ...  0.59846312 -0.20847595
   0.25736581]]
(640, 350)
(160, 350)
(350, 350)
0.6365796819833875
0.5523548157565996


In [8]:
from sklearn.utils import _safe_indexing
class AdaBoost:
    def __init__(self):
        self.stumps = None
        self.stump_weights = None
        self.errors = None
        self.sample_weights = None

    def fit(self, x_train, y_label, num_of_iterations):
      
      x = np.array(x_train)
      y =  np.array(y_label)
      num_samples = x.shape[0]
      #print(y.shape)

      self.sample_weights = np.zeros(shape=(num_of_iterations, num_samples))
      self.stumps = np.zeros(shape=num_of_iterations,dtype=object)
      self.stump_weights = np.zeros(shape=num_of_iterations)
      self.errors = np.zeros(shape=num_of_iterations)

      self.sample_weights[0] = np.ones(shape=num_samples) / num_samples
      
      for iters in range(num_of_iterations):
        current_sample_weights = self.sample_weights[iters]
        bootstrap_idx = np.random.choice(np.arange(num_samples),size=num_samples, replace=True, p=current_sample_weights)
        #print(bootstrap_idx)
        x = _safe_indexing(x, bootstrap_idx)
        y = _safe_indexing(y, bootstrap_idx)
        #print(current_sample_weights.shape)
        Decision_stump = DecisionTreeClassifier(max_depth=1)
        Decision_stump = Decision_stump.fit(x, y, sample_weight=current_sample_weights)
        y_prediction = Decision_stump.predict(x)
        Decision_stump_acc=accuracy_score(y,y_prediction)*100
        #print("The training accuracy for decision tree model is: ",Decision_stump_acc)
        #print(classification_report(y,y_prediction))
        incorrect = y_prediction != y
        #print(incorrect.shape)
        estimator_error = np.mean(np.average(incorrect, weights=current_sample_weights, axis=0))
        #print(estimator_error)
        if estimator_error <= 0.0:
          break
        elif estimator_error >= 0.5:
          continue
        #error = current_sample_weights[(y_prediction != y)].sum()
        #print(error)
        #stump_weight = np.log((1 - estimator_error) / estimator_error) / 2
        estimator_weight = ( np.log((1.0 - estimator_error) / estimator_error)) / 2
        #print(estimator_weight)
        new_sample_weights = current_sample_weights.copy()
        if np.array_equal(y_prediction,y):
          new_sample_weights = (current_sample_weights* np.exp(estimator_weight*np.asarray(y_prediction).astype(int)*y.astype(int)))
        else:
          new_sample_weights = (current_sample_weights * np.exp(-estimator_weight*np.asarray(y_prediction).astype(int)*y.astype(int)))
        new_sample_weights /= new_sample_weights.sum()

        if iters+1 < num_of_iterations:
          self.sample_weights[iters+1] = new_sample_weights
        
        self.stumps[iters] = Decision_stump
        self.stump_weights[iters] = estimator_weight
        self.errors[iters] = estimator_error
        #print(new_sample_weights)
        #print(new_sample_weights.shape)
        #print(type(y))
        #print(x)
        #print(y) 

      self.stumps = self.stumps[self.stumps != 0]
      self.stump_weights = self.stump_weights[self.stump_weights != 0]
      #print(self.stumps.shape)
      return self

    def predict(self, X):
      stump_preds = np.array([stump.predict(X) for stump in self.stumps])
      #print(stump_preds.shape,type(stump_preds[0]))
      #print(stump_preds[0].astype(int))
      #print(self.stump_weights.shape,type(self.stump_weights[0]))
      #for i in stump_preds:
      #  print(i)
      #for i in self.stump_weights:
      #  print(i)
      value = np.dot(self.stump_weights, stump_preds.astype(int))
      #print(value,"123")
      return np.sign(value)

In [9]:
adaboost = AdaBoost()
adaboost_train = adaboost.fit(train_svd,y_train,100)
#print(test_svd.shape)
y_predict = adaboost_train.predict(test_svd)

In [10]:
print(y_predict.astype(int))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]


In [11]:
acc=accuracy_score(np.asarray(y_test).astype(int),y_predict.astype(int))*100
print("The training accuracy for decision tree model is: ",acc)
print(classification_report(np.asarray(y_test).astype(int),y_predict.astype(int)))

The training accuracy for decision tree model is:  92.5
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       146
           1       1.00      0.14      0.25        14

    accuracy                           0.93       160
   macro avg       0.96      0.57      0.61       160
weighted avg       0.93      0.93      0.90       160



In [12]:
y_predict1 = adaboost_train.predict(test_sparse_svd)

In [13]:
print(y_predict1.astype(int))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [14]:
with open("HW3_result1.txt","w") as f:
  for s in y_predict1.astype(int):
    f.write(str(s)+"\n")

In [15]:
adaboost = AdaBoost()
adaboost_train = adaboost.fit(train_svd1, y_train,100)
#print(test_svd.shape)
y_predict2 = adaboost_train.predict(test_svd1)

In [16]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [17]:
classifiers_list = {
    "Adaboost" : AdaBoost()
}
classifiers_count = len(classifiers_list.keys())
df_results = pd.DataFrame(data=np.zeros(shape=(classifiers_count,5)), columns = ['classifier', 'Recall', 'F1', 'Precision', 'Accuracy'])
iteratins = 100
for c_name, classifier in classifiers_list.items():
  classifier.fit(train_svd, y_train,iteratins)
  prediction = []
  prediction = classifier.predict(test_svd)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_svd, y_train, cv=cv1,scoring ='f1_macro')
  print ('Classifier+Smote + SVD', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction.astype(int)))

  classifier.fit(train_pca, y_train,iteratins)
  prediction1 = []
  prediction1 = classifier.predict(test_pca)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_pca, y_train, cv=cv1)
  print ('Classifier +Smote + PCA', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction1.astype(int)))

  classifier.fit(train_sparse_sm_1, drug_active_sm_1,iteratins)
  prediction2 = []
  prediction2 = classifier.predict(test_sparse_1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier,train_sparse_sm_1 , drug_active_sm_1, cv=cv1)
  print ('Classifier + Chi', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction2.astype(int)))

  classifier.fit(train_svd1, y_train,iteratins)
  prediction3 = []
  prediction3 = classifier.predict(test_svd1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_svd1, y_train, cv=cv1)
  print ('Classifier + SVD', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction3.astype(int)))
  
  classifier.fit(train_pca1, y_train,iteratins)
  prediction4 = []
  prediction4 = classifier.predict(test_pca1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
  print ('Classifier + PCA', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))

  classifier.fit(train_sparse_1, y_train,iteratins)
  prediction4 = []
  prediction4 = classifier.predict(test_sparse_1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
  print ('Classifier + PCA', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))

Classifier+Smote + SVD Adaboost
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       146
           1       0.57      0.29      0.38        14

    accuracy                           0.92       160
   macro avg       0.75      0.63      0.67       160
weighted avg       0.90      0.92      0.91       160

Classifier +Smote + PCA Adaboost
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       146
           1       1.00      0.14      0.25        14

    accuracy                           0.93       160
   macro avg       0.96      0.57      0.61       160
weighted avg       0.93      0.93      0.90       160

Classifier + Chi Adaboost
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       146
           1       0.69      0.64      0.67        14

    accuracy                           0.94       160
   macro avg       0.83      0.81    

In [18]:
adaboost = AdaBoost()
adaboost_train = adaboost.fit(train_sparse_1, y_train,iteratins)
#print(test_svd.shape)
y_predict_1 = adaboost_train.predict(test_sparse_final)

In [19]:
print(y_predict_1.astype(int).shape)

(350,)


In [20]:
with open("HW3_result_7.txt","w") as f:
  for s in y_predict_1.astype(int):
    f.write(str(s)+"\n")

In [51]:
from sklearn.utils import _safe_indexing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

class AdaBoost1:
    def __init__(self):
        self.stumps = None
        self.stump_weights = None
        self.errors = None
        self.sample_weights = None

    def fit(self, x_train, y_label, num_of_iterations):
      
      x = np.array(x_train)
      y =  np.array(y_label)
      num_samples = x.shape[0]
      #print(y.shape)

      self.sample_weights = np.zeros(shape=(num_of_iterations, num_samples))
      self.stumps = np.zeros(shape=num_of_iterations,dtype=object)
      self.stump_weights = np.zeros(shape=num_of_iterations)
      self.errors = np.zeros(shape=num_of_iterations)

      self.sample_weights[0] = np.ones(shape=num_samples) / num_samples
      
      for iters in range(num_of_iterations):
        print(iters)
        current_sample_weights = self.sample_weights[iters]
        bootstrap_idx = np.random.choice(np.arange(num_samples),size=num_samples, replace=True, p=current_sample_weights)
        #print(bootstrap_idx)
        x = _safe_indexing(x, bootstrap_idx)
        y = _safe_indexing(y, bootstrap_idx)
        #print(current_sample_weights.shape)
        Decision_stump = DecisionTreeClassifier(max_depth = 2)
        Decision_stump = Decision_stump.fit(x, y, sample_weight=current_sample_weights)
        y_prediction = Decision_stump.predict(x)
        Decision_stump_acc=accuracy_score(y,y_prediction)*100
        #print("The training accuracy for decision tree model is: ",Decision_stump_acc)
        #print(classification_report(y,y_prediction))
        incorrect = y_prediction != y
        #print(incorrect.shape)
        estimator_error = np.mean(np.average(incorrect, weights=current_sample_weights, axis=0))
        #print(estimator_error)
        if estimator_error <= 0.0:
          break
        elif estimator_error >= 0.5:
          continue
        estimator_weight = ( np.log((1.0 - estimator_error) / estimator_error)) / 2
        #print(estimator_weight)
        new_sample_weights = current_sample_weights.copy()
        new_sample_weights = (current_sample_weights)* np.exp(-estimator_weight*np.asarray(y_prediction).astype(int)*y.astype(int))
        new_sample_weights /= new_sample_weights.sum()

        if iters+1 < num_of_iterations:
          self.sample_weights[iters+1] = new_sample_weights
        
        self.stumps[iters] = Decision_stump
        self.stump_weights[iters] = estimator_weight
        self.errors[iters] = estimator_error
        #print(new_sample_weights)
        #print(new_sample_weights.shape)
        #print(type(y))
        #print(x)
        #print(y) 

      self.stumps = self.stumps[self.stumps != 0]
      self.stump_weights = self.stump_weights[self.stump_weights != 0]
      #print(self.stumps.shape)
      return self

    def predict(self, X):
      stump_preds = np.array([stump.predict(X) for stump in self.stumps])
      #print(stump_preds.shape,type(stump_preds[0]))
      #print(stump_preds[0].astype(int))
      #print(self.stump_weights.shape,type(self.stump_weights[0]))
      #for i in stump_preds:
      #  print(i)
      #for i in self.stump_weights:
      #  print(i)
      value = np.dot(self.stump_weights, stump_preds.astype(int))
      #print(value,"123")
      return np.sign(value)

In [64]:
#from sklearn.linear_model import LogisticRegression
classifiers_list = {
    "AdaBoost1" : AdaBoost1()
}
classifiers_count = len(classifiers_list.keys())
df_results = pd.DataFrame(data=np.zeros(shape=(classifiers_count,5)), columns = ['classifier', 'Recall', 'F1', 'Precision', 'Accuracy'])
iteratins = 100
for c_name, classifier in classifiers_list.items():
  classifier.fit(train_svd, np.asarray(y_train).astype(int),iteratins)
  prediction = []
  prediction = classifier.predict(test_svd)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_svd, y_train, cv=cv1,scoring ='f1_macro')
  print ('Classifier+Smote + SVD', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction.astype(int)))

  classifier.fit(train_pca, y_train,iteratins)
  prediction1 = []
  prediction1 = classifier.predict(test_pca)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_pca, y_train, cv=cv1)
  print ('Classifier +Smote + PCA', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction1.astype(int)))

  classifier.fit(train_sparse_sm_1, drug_active_sm_1,iteratins)
  prediction2 = []
  prediction2 = classifier.predict(test_sparse_1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier,train_sparse_sm_1 , drug_active_sm_1, cv=cv1)
  print ('Classifier + Chi+ SMOTE', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction2.astype(int)))

  classifier.fit(train_svd1, y_train,iteratins)
  prediction3 = []
  prediction3 = classifier.predict(test_svd1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_svd1, y_train, cv=cv1)
  print ('Classifier + SVD', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction3.astype(int)))
  
  classifier.fit(train_pca1, y_train,iteratins)
  prediction4 = []
  prediction4 = classifier.predict(test_pca1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
  print ('Classifier + PCA', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))

  classifier.fit(train_sparse_1, y_train,iteratins)
  prediction4 = []
  prediction4 = classifier.predict(test_sparse_1)
  cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
  #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
  print ('Classifier + Chi', c_name)
  #print ('Cross validation', scores)
  print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))


0
1
2
3
4
5
Classifier+Smote + SVD AdaBoost1
              precision    recall  f1-score   support

           0       0.92      0.99      0.95       146
           1       0.50      0.14      0.22        14

    accuracy                           0.91       160
   macro avg       0.71      0.56      0.59       160
weighted avg       0.89      0.91      0.89       160

0
1
2
3
4
Classifier +Smote + PCA AdaBoost1
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       146
           1       0.75      0.64      0.69        14

    accuracy                           0.95       160
   macro avg       0.86      0.81      0.83       160
weighted avg       0.95      0.95      0.95       160

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


In [68]:
adaboost = AdaBoost1()
adaboost_train = adaboost.fit(train_sparse_1, y_train,iteratins)
#print(test_svd.shape)
y_predict_1 = adaboost_train.predict(test_sparse_final)

0
1
2
3
4


In [69]:
print(y_predict_1.astype(int).shape)

(350,)


In [70]:
with open("HW3_result_9.txt","w") as f:
  for s in y_predict_1.astype(int):
    f.write(str(s)+"\n")

In [86]:
from sklearn.utils import _safe_indexing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

class AdaBoost2:
    def __init__(self):
        self.stumps = None
        self.stump_weights = None
        self.errors = None
        self.sample_weights = None

    def fit(self, x_train, y_label, num_of_iterations):
      
      x = np.array(x_train)
      y =  np.array(y_label)
      num_samples = x.shape[0]
      #print(y.shape)

      self.sample_weights = np.zeros(shape=(num_of_iterations, num_samples))
      self.stumps = np.zeros(shape=num_of_iterations,dtype=object)
      self.stump_weights = np.zeros(shape=num_of_iterations)
      self.errors = np.zeros(shape=num_of_iterations)

      self.sample_weights[0] = np.ones(shape=num_samples) / num_samples
      
      for iters in range(num_of_iterations):
        print(iters)
        current_sample_weights = self.sample_weights[iters]
        bootstrap_idx = np.random.choice(np.arange(num_samples),size=num_samples, replace=True, p=current_sample_weights)
        #print(bootstrap_idx)
        x_1 = _safe_indexing(x, bootstrap_idx)
        y_1 = _safe_indexing(y, bootstrap_idx)
        #print(current_sample_weights.shape)
        Decision_stump = DecisionTreeClassifier(max_depth = 2)
        Decision_stump = Decision_stump.fit(x_1, y_1, sample_weight=current_sample_weights)
        y_prediction = Decision_stump.predict(x)
        Decision_stump_acc=accuracy_score(y,y_prediction)*100
        #print("The training accuracy for decision tree model is: ",Decision_stump_acc)
        #print(classification_report(y,y_prediction))
        incorrect = y_prediction != y
        #print(incorrect.shape)
        estimator_error = np.mean(np.average(incorrect, weights=current_sample_weights, axis=0))
        #print(estimator_error)
        if estimator_error <= 0.0:
          break
        elif estimator_error >= 0.5:
          continue
        estimator_weight = ( np.log((1.0 - estimator_error) / estimator_error)) / 2
        #print(estimator_weight)
        new_sample_weights = current_sample_weights.copy()
        new_sample_weights = (current_sample_weights)* np.exp(-estimator_weight*np.asarray(y_prediction).astype(int)*y.astype(int))
        new_sample_weights /= new_sample_weights.sum()

        if iters+1 < num_of_iterations:
          self.sample_weights[iters+1] = new_sample_weights
        
        self.stumps[iters] = Decision_stump
        self.stump_weights[iters] = estimator_weight
        self.errors[iters] = estimator_error
        #print(new_sample_weights)
        #print(new_sample_weights.shape)
        #print(type(y))
        #print(x)
        #print(y) 

      self.stumps = self.stumps[self.stumps != 0]
      self.stump_weights = self.stump_weights[self.stump_weights != 0]
      #print(self.stumps.shape)
      return self

    def predict(self, X):
      stump_preds = np.array([stump.predict(X) for stump in self.stumps])
      #print(stump_preds.shape,type(stump_preds[0]))
      #print(stump_preds[0].astype(int))
      #print(self.stump_weights.shape,type(self.stump_weights[0]))
      #for i in stump_preds:
      #  print(i)
      #for i in self.stump_weights:
      #  print(i)
      value = np.dot(self.stump_weights, stump_preds.astype(int))
      #print(value,"123")
      return np.sign(value)

In [91]:
#from sklearn.linear_model import LogisticRegression
classifiers_list = {
    "AdaBoost2" : AdaBoost2()
}
classifiers_count = len(classifiers_list.keys())
df_results = pd.DataFrame(data=np.zeros(shape=(classifiers_count,5)), columns = ['classifier', 'Recall', 'F1', 'Precision', 'Accuracy'])
iters = [50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200]
for c_name, classifier in classifiers_list.items():
  for iterations in iters:
    print("********************Iterations : %d ********************"%iterations)
    classifier.fit(train_svd, np.asarray(y_train).astype(int),iteratins)
    prediction = []
    prediction = classifier.predict(test_svd)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_svd, y_train, cv=cv1,scoring ='f1_macro')
    print ('Classifier+Smote + SVD', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction.astype(int)))

    classifier.fit(train_pca, y_train,iteratins)
    prediction1 = []
    prediction1 = classifier.predict(test_pca)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_pca, y_train, cv=cv1)
    print ('Classifier +Smote + PCA', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction1.astype(int)))

    classifier.fit(train_sparse_sm_1, drug_active_sm_1,iteratins)
    prediction2 = []
    prediction2 = classifier.predict(test_sparse_1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier,train_sparse_sm_1 , drug_active_sm_1, cv=cv1)
    print ('Classifier + Chi+ SMOTE', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction2.astype(int)))

    classifier.fit(train_svd1, y_train,iteratins)
    prediction3 = []
    prediction3 = classifier.predict(test_svd1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_svd1, y_train, cv=cv1)
    print ('Classifier + SVD', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction3.astype(int)))
  
    classifier.fit(train_pca1, y_train,iteratins)
    prediction4 = []
    prediction4 = classifier.predict(test_pca1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
    print ('Classifier + PCA', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))

    classifier.fit(train_sparse_1, y_train,iteratins)
    prediction4 = []
    prediction4 = classifier.predict(test_sparse_1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
    print ('Classifier + Chi', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))

********************Iterations : 50 ********************
Classifier+Smote + SVD AdaBoost2
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       146
           1       1.00      0.29      0.44        14

    accuracy                           0.94       160
   macro avg       0.97      0.64      0.71       160
weighted avg       0.94      0.94      0.92       160

Classifier +Smote + PCA AdaBoost2
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       146
           1       1.00      0.64      0.78        14

    accuracy                           0.97       160
   macro avg       0.98      0.82      0.88       160
weighted avg       0.97      0.97      0.97       160

Classifier + Chi+ SMOTE AdaBoost2
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       146
           1       0.57      0.93      0.70        14

    accuracy            

In [100]:
from sklearn.utils import _safe_indexing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

class AdaBoost3:
    def __init__(self):
        self.stumps = None
        self.stump_weights = None
        self.errors = None
        self.sample_weights = None

    def fit(self, x_train, y_label, num_of_iterations):
      
      x = np.array(x_train)
      y =  np.array(y_label)
      num_samples = x.shape[0]
      #print(y.shape)

      self.sample_weights = np.zeros(shape=(num_of_iterations, num_samples))
      self.stumps = np.zeros(shape=num_of_iterations,dtype=object)
      self.stump_weights = np.zeros(shape=num_of_iterations)
      self.errors = np.zeros(shape=num_of_iterations)

      self.sample_weights[0] = np.ones(shape=num_samples) / num_samples
      
      for iters in range(num_of_iterations):
        #print(iters)
        current_sample_weights = self.sample_weights[iters]
        bootstrap_idx = np.random.choice(np.arange(num_samples),size=num_samples, replace=True, p=current_sample_weights)
        #print(bootstrap_idx)
        x_1 = _safe_indexing(x, bootstrap_idx)
        y_1 = _safe_indexing(y, bootstrap_idx)
        #print(current_sample_weights.shape)
        Decision_stump = LogisticRegression()
        Decision_stump = Decision_stump.fit(x_1, y_1, sample_weight=current_sample_weights)
        y_prediction = Decision_stump.predict(x)
        Decision_stump_acc=accuracy_score(y,y_prediction)*100
        #print("The training accuracy for decision tree model is: ",Decision_stump_acc)
        #print(classification_report(y,y_prediction))
        incorrect = y_prediction != y
        #print(incorrect.shape)
        estimator_error = np.mean(np.average(incorrect, weights=current_sample_weights, axis=0))
        #print(estimator_error)
        if estimator_error <= 0.0:
          break
        elif estimator_error >= 0.5:
          continue
        estimator_weight = ( np.log((1.0 - estimator_error) / estimator_error)) / 2
        #print(estimator_weight)
        new_sample_weights = current_sample_weights.copy()
        if np.array_equal(y_prediction,y):
          new_sample_weights = (current_sample_weights* np.exp(estimator_weight*0))
        else:
          new_sample_weights = (current_sample_weights * np.exp(-estimator_weight))
        new_sample_weights /= new_sample_weights.sum()

        if iters+1 < num_of_iterations:
          self.sample_weights[iters+1] = new_sample_weights
        
        self.stumps[iters] = Decision_stump
        self.stump_weights[iters] = estimator_weight
        self.errors[iters] = estimator_error
        #print(new_sample_weights)
        #print(new_sample_weights.shape)
        #print(type(y))
        #print(x)
        #print(y) 

      self.stumps = self.stumps[self.stumps != 0]
      self.stump_weights = self.stump_weights[self.stump_weights != 0]
      #print(self.stumps.shape)
      return self

    def predict(self, X):
      stump_preds = np.array([stump.predict(X) for stump in self.stumps])
      #print(stump_preds.shape,type(stump_preds[0]))
      #print(stump_preds[0].astype(int))
      #print(self.stump_weights.shape,type(self.stump_weights[0]))
      #for i in stump_preds:
      #  print(i)
      #for i in self.stump_weights:
      #  print(i)
      value = np.dot(self.stump_weights, stump_preds.astype(int))
      #print(value,"123")
      return np.sign(value)

In [102]:
adaboost = AdaBoost3()
adaboost_train = adaboost.fit(train_sparse_sm_1, drug_active_sm_1,10)
#print(test_svd.shape)
y_predict_1 = adaboost_train.predict(test_sparse_final)

In [103]:
with open("HW3_result_11.txt","w") as f:
  for s in y_predict_1.astype(int):
    f.write(str(s)+"\n")

In [104]:
#from sklearn.linear_model import LogisticRegression
classifiers_list = {
    "AdaBoost3" : AdaBoost3()
}
classifiers_count = len(classifiers_list.keys())
df_results = pd.DataFrame(data=np.zeros(shape=(classifiers_count,5)), columns = ['classifier', 'Recall', 'F1', 'Precision', 'Accuracy'])
iters = [50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200]
for c_name, classifier in classifiers_list.items():
  for iterations in iters:
    print("********************Iterations : %d ********************"%iterations)
    classifier.fit(train_svd, np.asarray(y_train).astype(int),iteratins)
    prediction = []
    prediction = classifier.predict(test_svd)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_svd, y_train, cv=cv1,scoring ='f1_macro')
    print ('Classifier+Smote + SVD', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction.astype(int)))

    classifier.fit(train_pca, y_train,iteratins)
    prediction1 = []
    prediction1 = classifier.predict(test_pca)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_pca, y_train, cv=cv1)
    print ('Classifier +Smote + PCA', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction1.astype(int)))

    classifier.fit(train_sparse_sm_1, drug_active_sm_1,iteratins)
    prediction2 = []
    prediction2 = classifier.predict(test_sparse_1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier,train_sparse_sm_1 , drug_active_sm_1, cv=cv1)
    print ('Classifier + Chi+ SMOTE', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction2.astype(int)))

    classifier.fit(train_svd1, y_train,iteratins)
    prediction3 = []
    prediction3 = classifier.predict(test_svd1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_svd1, y_train, cv=cv1)
    print ('Classifier + SVD', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction3.astype(int)))
  
    classifier.fit(train_pca1, y_train,iteratins)
    prediction4 = []
    prediction4 = classifier.predict(test_pca1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
    print ('Classifier + PCA', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))

    classifier.fit(train_sparse_1, y_train,iteratins)
    prediction4 = []
    prediction4 = classifier.predict(test_sparse_1)
    cv1 = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    #scores = cross_val_score(classifier, train_pca1, y_train, cv=cv1)
    print ('Classifier + Chi', c_name)
    #print ('Cross validation', scores)
    print(classification_report(np.asarray(y_test).astype(int),prediction4.astype(int)))

********************Iterations : 50 ********************
Classifier+Smote + SVD AdaBoost3
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       146
           1       0.47      0.64      0.55        14

    accuracy                           0.91       160
   macro avg       0.72      0.79      0.75       160
weighted avg       0.92      0.91      0.91       160

Classifier +Smote + PCA AdaBoost3
              precision    recall  f1-score   support

           0       0.98      0.84      0.91       146
           1       0.34      0.86      0.49        14

    accuracy                           0.84       160
   macro avg       0.66      0.85      0.70       160
weighted avg       0.93      0.84      0.87       160

Classifier + Chi+ SMOTE AdaBoost3
              precision    recall  f1-score   support

           0       0.97      0.96      0.97       146
           1       0.62      0.71      0.67        14

    accuracy            